# Pytorch Basics 00

In [10]:
from __future__ import print_function, division
import torch
import torchvision
import torch.nn as nn
import numpy as np
import torch.utils.data as data
import torchvision.transforms as transforms
import torchvision.datasets as dsets
from torch.autograd import Variable

## Autograd Example 1


Create Tensors

In [11]:
x = Variable(torch.Tensor([1]), requires_grad=True)
w = Variable(torch.Tensor([2]), requires_grad=True)
b = Variable(torch.Tensor([3]), requires_grad=True)

Build a computational graph

In [12]:
y = w * x + b 

Compute gradients

In [13]:
y.backward()

print(x.grad)
print(w.grad)
print(b.grad)

Variable containing:
 2
[torch.FloatTensor of size 1]

Variable containing:
 1
[torch.FloatTensor of size 1]

Variable containing:
 1
[torch.FloatTensor of size 1]



## Autograd Example 2

In [22]:
x = Variable(torch.randn(5,4))
y = Variable(torch.randn(5,2))

Build a linear layer

In [23]:
linear = nn.Linear(4,2)
print('w: ', linear.weight)
print('b: ', linear.bias)

w:  Parameter containing:
-0.4776  0.3268 -0.1731 -0.0968
 0.1563 -0.0338  0.1136 -0.2984
[torch.FloatTensor of size 2x4]

b:  Parameter containing:
 0.3847
 0.2596
[torch.FloatTensor of size 2]



Build Loss and Optimizer

In [24]:
criterion = nn.MSELoss()
optimizer = torch.optim.SGD(linear.parameters(), lr=0.01)

Forward propagation

In [25]:
pred = linear(x)

Compute loss

In [26]:
loss = criterion(pred, y)
print('loss: ', loss.data[0])

loss:  2.32674264908


Backpropagation

In [27]:
loss.backward()

Print out the gradients

In [28]:
print('dL/dw: ', linear.weight.grad)
print('dL/db: ', linear.bias.grad)

dL/dw:  Variable containing:
-1.9121 -0.5483 -0.7888 -0.3170
 1.1845  0.4778  0.6665 -0.3684
[torch.FloatTensor of size 2x4]

dL/db:  Variable containing:
-0.0051
 1.0524
[torch.FloatTensor of size 2]



1 step optimization (gradient descent)

In [29]:
optimizer.step()

You can also do optimization at the low level as shown below

In [30]:
#linear.weight.data.sub_(0.01 * linear.weight.grad.data)
#linear.bias.data.sub_(0.01 * linear.bias.grad.data)

Print out the loss after optimization

In [31]:
pred = linear(x)
loss = criterion(pred, y)
print('loss after 1 step optimization: ', loss.data[0])

loss after 1 step optimization:  2.24785733223


## Loading data from numpy

In [32]:
a = np.array([[1,2], [3,4]])
b = torch.from_numpy(a) # convert to a tensor
c = b.numpy() # convert to numpy array

## Implementing the input pipeline

Download and construct dataset

In [33]:
train_dataset = dsets.CIFAR10(root='../data/',
                              train=True,
                              transform=transforms.ToTensor(),
                              download=True)

Files already downloaded and verified


Select one data pair (read data from disk)

In [34]:
image, label = train_dataset[0]
print(image.size())
print(label)

torch.Size([3, 32, 32])
6


Data Loader (this provides queue and thread in a very simple way)

In [35]:
train_loader = torch.utils.data.DataLoader(dataset=train_dataset,
                                           batch_size=100,
                                           shuffle=True,
                                           num_workers=2)

When iteration starts, queue and thread start to load dataset from files

In [36]:
data_iter = iter(train_loader)

Mini-batch images and labels

In [37]:
images, labels = data_iter.next()

Actual usage of data loader is as below

In [38]:
for images, labels in train_loader:
    #training code
    pass 

## Input pipeline for custom dataset

You should build custom dataset as below

In [39]:
class CustomDataset(data.Dataset):
    def __init__(self):
        #TODO
        #1. Initialize file path or list of file names
        pass
    def __getitem__(self, index):
        #TODO
        #1.Read one data from file (eg. using numpy.fromfile, PIL.Image.open)
        #2.Preprocess the data (eg. torchvision.Transform)
        #3.Return a data pair (eg. image and label)
        pass
    def __len__(self):
        #Change 0 to the size of your dataset
        return 0

Then you can just use prebuilt torch's data loader

In [40]:
custom_dataset = CustomDataset()
train_loader = torch.utils.data.DataLoader(dataset=custom_dataset,
                                           batch_size=100,
                                           shuffle=True,
                                           num_workers=2)

## Using pretrained model

Download and load pretrained resnet

In [41]:
resnet = torchvision.models.resnet18(pretrained=True)

If you want to finetune only top layer of the model

In [42]:
for param in resnet.parameters():
    param.requires_grad = False

Replace top layer for finetuning

In [43]:
resnet.fc = nn.Linear(resnet.fc.in_features, 100) #100 is an example

For test

In [44]:
images = Variable(torch.randn(10, 3, 256, 256))
outputs = resnet(images)
print(outputs.size())

torch.Size([10, 100])


## Save and load the model

Save and load the entire model

In [46]:
torch.save(resnet, 'model.pkl')
model = torch.load('model.pkl')

Save and load only the model parameters (recommended)

In [47]:
torch.save(resnet.state_dict(), 'params.pkl')
resnet.load_state_dict(torch.load('params.pkl'))